In [ ]:
import warnings
!pip install datasets
warnings.simplefilter("ignore")
import torch.nn as nn
import torch
import torch.nn.functional as F
import math,copy,re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import gc
import spacy
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
from torchtext.vocab import vocab
from collections import Counter
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset
import torch
import torch.nn.functional as F
print(torch.__version__)


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic=True

set_seed(42)




class Embeddings(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(Embeddings, self).__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.embed_layer = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.embed_dim)

    def forward(self, x):
        out = self.embed_layer(x)
        return out



class PositionalEmbedding(nn.Module):
    def __init__(self, max_seq_len, embed_dim):
        super(PositionalEmbedding, self).__init__()
        self.embed_dim = embed_dim
        self.max_seq_len = max_seq_len

        pe = torch.zeros((self.max_seq_len, self.embed_dim))

        for pos in range(self.max_seq_len):
            for i in range(0, self.embed_dim, 2):
                pe[pos, i] = math.sin(pos / (10000**(i/self.embed_dim)))
                pe[pos, i+1] = math.cos(pos / (10000**(i/self.embed_dim)))

        pe = pe.unsqueeze(0)

        self.register_buffer('pe', pe)

    def forward(self, inp):
        inp = inp*math.sqrt(self.embed_dim)
        seq_len = inp.size(1)
        inp = inp + torch.autograd.Variable(self.pe[:, :seq_len], requires_grad=False)
        return inp




class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim=512, n_heads=8):
        super(MultiHeadAttention, self).__init__()

        self.n_heads = n_heads
        self.embed_dim = embed_dim

        self.head_dim = int(self.embed_dim/self.n_heads)

        self.query_matrix = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.key_matrix = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.value_matrix = nn.Linear(self.head_dim, self.head_dim, bias=False)

        self.out = nn.Linear(embed_dim, embed_dim)


    def forward(self, key, query, value, mask=None):
        batch_size = key.size(0)
        seq_len = key.size(1)

        seq_len_query = query.size(1)

        key = key.view(batch_size, seq_len, self.n_heads, self.head_dim)
        query = query.view(batch_size, seq_len_query, self.n_heads, self.head_dim)
        value = value.view(batch_size, seq_len, self.n_heads, self.head_dim)

        k = self.key_matrix(key)
        q = self.query_matrix(query)
        v = self.value_matrix(value)

        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)

        k_adj = k.transpose(-1,-2)

        # prdt = torch.einsum("bhqd,bhdk->bhqk", q, k_adj)
        prdt = torch.matmul(q, k_adj)

        if mask is not None:
            prdt = prdt.masked_fill(mask==0, float("-1e20"))

        prdt = prdt/math.sqrt(self.embed_dim)
        prdt = F.softmax(prdt, dim=-1)

        # attention = torch.einsum("bhqk,bhkd->bhqd", prdt, v)
        attention = torch.matmul(prdt, v)

        concat = attention.transpose(1,2).contiguous().view(batch_size, seq_len_query, self.head_dim*self.n_heads)

        out = self.out(concat)

        return out


class TransformerBlock(nn.Module):
    def __init__(self, embed_dim=512, n_heads=8, expansion_factor=4):
        super(TransformerBlock, self).__init__()

        self.embed_dim = embed_dim
        self.n_heads = n_heads
        self.expansion_factor = expansion_factor

        self.multiheadattention = MultiHeadAttention(self.embed_dim, self.n_heads)

        self.norm1 = nn.LayerNorm(self.embed_dim)
        self.dropout1 = nn.Dropout(0.1)

        self.feed_forward = nn.Sequential(
            nn.Linear(self.embed_dim, self.embed_dim*self.expansion_factor),
            nn.ReLU(),
            nn.Linear(self.embed_dim*self.expansion_factor, self.embed_dim)
            )
        self.norm2 = nn.LayerNorm(self.embed_dim)
        self.dropout2 = nn.Dropout(0.1)


    def forward(self, key, query, value, mask=None):
        attention_out = self.multiheadattention(key, query, value, mask)
        attention_residual_out = attention_out + query
        norm1_out = self.dropout1(self.norm1(attention_residual_out))

        feed_forward_out = self.feed_forward(norm1_out)
        feed_forward_residual_out = feed_forward_out + norm1_out
        norm2_out = self.dropout2(self.norm2(feed_forward_residual_out))

        return norm2_out

class TransformerEncoder(nn.Module):
    def __init__(self, max_seq_len, vocab_size, embed_size=512, num_layers=6, n_heads=8, expansion_factor=4):
        super(TransformerEncoder, self).__init__()

        self.embedding_layer = Embeddings(vocab_size, embed_size)
        self.positional_embeddings = PositionalEmbedding(max_seq_len, embed_size)

        self.layers = nn.ModuleList([
            TransformerBlock(embed_size, n_heads, expansion_factor) for i in range(num_layers)
        ])

    def forward(self, x, mask=None):
        embed = self.embedding_layer(x)
        out = self.positional_embeddings(embed)

        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out


class DecoderBlock(nn.Module):
    def __init__(self, embed_dim=512, n_heads=8, expansion_factor=4):
        super(DecoderBlock, self).__init__()

        self.embed_dim = embed_dim
        self.n_heads = n_heads
        self.expansion_factor = expansion_factor

        self.transformer_block = TransformerBlock(embed_dim, n_heads, expansion_factor)
        self.attention = MultiHeadAttention(embed_dim, n_heads)
        self.norm = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(0.1)

    def forward(self, key, value, x, tgt_mask, src_mask=None):
        attention = self.attention(x, x, x, tgt_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(key, query, value, src_mask)
        return out

class TransformerDecoder(nn.Module):
    def __init__(self, max_seq_len, target_vocab_size, embed_dim=512, num_layers=6, expansion_factor=4, n_heads=8):
        super(TransformerDecoder, self).__init__()

        self.word_embedding = Embeddings(target_vocab_size, embed_dim)
        self.position_embedding = PositionalEmbedding(max_seq_len, embed_dim)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_dim, expansion_factor=expansion_factor, n_heads=n_heads)
                for _ in range(num_layers)
            ]

        )
        self.fc_out = nn.Linear(embed_dim, target_vocab_size)

    def forward(self, x, enc_out, tgt_mask, src_mask=None):
        embed = self.word_embedding(x)
        x = self.position_embedding(embed)

        for layer in self.layers:
            x = layer(enc_out, enc_out, x, tgt_mask, src_mask)

        logits = self.fc_out(x)

        return logits

class Transformer(nn.Module):
    def __init__(self, embed_dim, src_vocab_size, target_vocab_size, max_seq_length, num_layers=6, expansion_factor=4, n_heads=8, device='cpu'):
        super(Transformer, self).__init__()

        self.src_pad_idx = -1
        self.tgt_pad_idx = -1
        self.device = device

        self.encoder = TransformerEncoder(max_seq_length,
                                          src_vocab_size,
                                          embed_dim,
                                          num_layers=num_layers,
                                          expansion_factor=expansion_factor,
                                          n_heads=n_heads)

        self.decoder = TransformerDecoder(max_seq_length,
                                          target_vocab_size,
                                          embed_dim,
                                          num_layers=num_layers,
                                          expansion_factor=expansion_factor,
                                          n_heads=n_heads)


    def make_tgt_mask(self, tgt):
        batch_size, tgt_len = tgt.shape
        tgt_mask = torch.tril(torch.ones((tgt_len, tgt_len))).expand(
            batch_size, 1, tgt_len, tgt_len
        ).bool()
        tgt_pad_mask = (tgt.cpu() != self.tgt_pad_idx).unsqueeze(1).unsqueeze(2).bool()
        tgt_mask = tgt_mask & tgt_pad_mask
        return tgt_mask.to(self.device)

    def make_pad_mask(self, inp, pad_idx):
        mask = (inp != pad_idx).unsqueeze(1).unsqueeze(2).bool()
        return mask.to(self.device)

    def forward(self, src, tgt):
        tgt_mask = self.make_tgt_mask(tgt)
        src_mask = self.make_pad_mask(src, self.src_pad_idx)
        enc_out = self.encoder(src)
        outputs = self.decoder(tgt, enc_out, tgt_mask, src_mask)
        return outputs


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
2.1.0+cu121


In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz
spacy_eng = spacy.load("en_core_sci_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 46.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 24.8 MB/s eta 0:00:00
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.5.1-py3-none-any.whl size=15870854 sha256=3fc8fb8e973149c4d2b202ce622c5c12335df8d166981d44da9021d97f138388
  Stored in directory: /root/.cache/pip/wheels/35/8f/33/4c20916692704167c1d78c3f11e7deff5b7c2ba4875eacf648
Successfully built en-core-sci-sm
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not

In [ ]:
path_to_gdrive_csv = '/content/drive/MyDrive/internal/mpng_cust/conditions_mapped_cons2.csv'

In [ ]:
def clean_input(x):
  stop_words = ['noc', 'nos', 'unknown_unit', 'comment', 'due', 'unspecified',
              'finding', 'null', 'none', 'unknown', 'xamfx', 'xamfc', 'histology',
                'if', 'in', 'org', 'xaxcg', 'xabzh', 'xaivs', 'xanwb']
  x = re.sub('[^\w\s]',' ', x.lower())
  result = ' '.join([word for word in x.split() if word not in (stop_words)])
  return result

In [ ]:
import pandas as pd
import re

In [ ]:
df = pd.read_csv(path_to_gdrive_csv)
df.TARGET_CONCEPT_NAME = df['TARGET_CONCEPT_NAME'].apply(lambda x: x.lower())

In [ ]:
df.SOURCE_CODE = df['SOURCE_CODE'].apply(clean_input)

In [ ]:
df = df[~df['SOURCE_CODE'].str.contains("no |not ")]

In [ ]:
trash_to_exclude = list(df[df['SOURCE_CODE'].str.startswith('xa')]['SOURCE_CODE'])

In [ ]:
trash_to_exclude

In [ ]:
for i in range(7): trash_to_exclude.pop(0)

In [ ]:
df = df[~df['SOURCE_CODE'].isin(trash_to_exclude)]

In [ ]:
df.shape

(116156, 2)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df,test_size=0.15, random_state=42)

In [ ]:
X_test.to_csv('test.csv')

In [ ]:
X_train, X_test = train_test_split(df,test_size=0.15, random_state=42)
spacy_eng = spacy.load("en_core_sci_sm")


from datasets.dataset_dict import DatasetDict
from datasets import Dataset

dat = {'train':Dataset.from_dict({'label':X_train.TARGET_CONCEPT_NAME,'text':X_train.SOURCE_CODE}),
       'test':Dataset.from_dict({'label':X_test.TARGET_CONCEPT_NAME,'text':X_test.SOURCE_CODE})
     }

dat1  = DatasetDict(dat)
train, test = dat1['train'], dat1['test']


def tokenizer_eng(text):
    return [tok.text for tok in spacy_eng.tokenizer(text)]

sou_counter = Counter()
tar_counter = Counter()
for data in tqdm(train):
    sou_counter.update(tokenizer_eng(data['text'].lower()))
    tar_counter.update(tokenizer_eng(data['label'].lower()))

sou_vocab = vocab(sou_counter, min_freq=2, specials=("<unk>", "<pad>", "<sos>", "<eos>"))
tar_vocab = vocab(tar_counter, min_freq=2, specials=("<unk>", "<pad>", "<sos>", "<eos>"))

100%|██████████| 98732/98732 [00:18<00:00, 5385.95it/s]


In [ ]:
sou_vocab.set_default_index(sou_vocab["<unk>"])
tar_vocab.set_default_index(tar_vocab["<unk>"])
print(f"Size of Source Vocab : {len(sou_vocab)}\n Size of Target Vocab : {len(tar_vocab)}")

text_transform_tar = lambda x: [tar_vocab['<sos>']] + [tar_vocab[token.lower()] for token in tokenizer_eng(x)] + [tar_vocab['<eos>']]
text_transform_sou = lambda x: [sou_vocab['<sos>']] + [sou_vocab[token.lower()] for token in tokenizer_eng(x)] + [sou_vocab['<eos>']]

def collate_batch(batch):
    src_list, tgt_list = [], []
    for data in batch:
        src_list.append(torch.tensor(text_transform_sou(data['text'])))
        tgt_list.append(torch.tensor(text_transform_tar(data['label'])))

    src_list = pad_sequence(src_list, padding_value=sou_vocab['<pad>']).T
    tgt_list = pad_sequence(tgt_list, padding_value=tar_vocab['<pad>']).T

    inp = {
        "src": src_list,
        "tgt": tgt_list
    }

    return inp


num_epochs = 30
batch_size = 16
learning_rate = 1e-3
weight_decay = 0.001
writer = SummaryWriter(f"runs/loss")

train_dataloader = DataLoader(train,
                              collate_fn=collate_batch,
                              shuffle=True,
                              batch_size=batch_size,
                              pin_memory=True)
test_dataloader = DataLoader(test,
                              collate_fn=collate_batch,
                              shuffle=False,
                              batch_size=batch_size,
                              pin_memory=True)

Size of Source Vocab : 8112
 Size of Target Vocab : 5437


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



transformer_model = Transformer(embed_dim=512,
                                src_vocab_size=len(sou_vocab),
                                target_vocab_size=len(tar_vocab),
                                max_seq_length=50,
                                num_layers=6,
                                expansion_factor=4,
                                n_heads=8,
                                device=device)
transformer_model.src_pad_idx = sou_vocab['<pad>']
transformer_model.tgt_pad_idx = tar_vocab['<pad>']

import math
total_steps = num_epochs*math.ceil(len(train)/batch_size)

optimizer = torch.optim.Adam(transformer_model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                               max_lr=learning_rate,
                                               total_steps=total_steps,
                                               pct_start=0.33,
                                               div_factor=1e3,
                                               final_div_factor=1e2)


import torch.nn as nn
criterion = nn.CrossEntropyLoss(ignore_index=tar_vocab['<pad>'])

In [ ]:
transformer_model = transformer_model.to(device)

In [ ]:
def translate_seq_beam_search(model, src, device, k=2, max_len=50):
    model.eval()

    src_mask = model.make_pad_mask(src, model.src_pad_idx)
    with torch.no_grad():
        enc_out = model.encoder(src, src_mask)

    # beam search

    candidates = [(torch.LongTensor([tar_vocab['<sos>']]), 0.0)]

    final_translations = []

    for a in range(max_len):

        input_batch = torch.concat([c[0].unsqueeze(0) for c in candidates], dim=0).to(device)

        if a>0:
            enc_out_repeat = enc_out.repeat(input_batch.shape[0], 1, 1)
        else:
            enc_out_repeat = enc_out


        with torch.no_grad():
            output = model.decoder(input_batch, enc_out_repeat, model.make_tgt_mask(input_batch), src_mask).detach().cpu()
        output[:, :, :2] = float("-1e20")
        output = output[:, -1, :]
        output = F.log_softmax(output, dim=-1)


        topk_output = torch.topk(output, k, dim=-1)
        topk_tokens = topk_output.indices
        topk_scores = topk_output.values


        new_seq = torch.concat([torch.concat([torch.vstack([c[0] for _ in range(k)]), topk_tokens[i].reshape(-1,1)], dim=-1) for i,c in enumerate(candidates)], dim=0)
        new_scores = torch.concat([c[1] + topk_scores[i] for i,c in enumerate(candidates)], dim=0)


        topk_new = torch.topk(new_scores, k=k).indices.tolist()

        new_candidates = []

        for i in range(k):
            if new_seq[topk_new[i]][-1] == tar_vocab["<eos>"] or a==max_len-1:
                final_translations.append((new_seq[topk_new[i]].tolist(), int(new_scores[topk_new[i]])))
            else:
                new_candidate = (new_seq[topk_new[i]], new_scores[topk_new[i]])
                new_candidates.append(new_candidate)


        if len(new_candidates) > 0:
            candidates = new_candidates
        else:
            break


    return final_translations
def translate_seq(model, src, device, max_len=50):
    model.eval()
    src_mask = model.make_pad_mask(src, model.src_pad_idx)
    with torch.no_grad():
        enc_src = model.encoder(src, src_mask)
    tgt_indexes = [tar_vocab["<sos>"]]
    for i in range(max_len):
        tgt_tensor = torch.LongTensor(tgt_indexes).unsqueeze(0).to(device)
        tgt_mask = model.make_tgt_mask(tgt_tensor)
        with torch.no_grad():
            output = model.decoder(tgt_tensor, enc_src, tgt_mask, src_mask)
        output[:, :, :2] = float("-1e20")  # cannot predict <unk>, <pad> token
        output = output[:, -1, :] # pick the last token
        output = F.softmax(output, dim=-1)
        pred_token = output.argmax(-1).item()
        tgt_indexes.append(pred_token)
        if pred_token == tar_vocab["<eos>"]:
            break
    return tgt_indexes



def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

class AvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()

    def reset(self):
        self.avg, self.sum, self.count = [0]*3

    def update(self, val, count=1):
        self.count += count
        self.sum += val * count
        self.avg = self.sum / self.count

    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text


In [ ]:
### training ###
step = 0
for epoch in range(1, num_epochs+1):

    print(f"[Epoch {epoch} / {num_epochs}]")

    checkpoint = {"state_dict": transformer_model.state_dict(), "optimizer": optimizer.state_dict()}
    torch.save(checkpoint, "/content/drive/MyDrive/internal/mpng_cust/my_checkpoint.pth.tar")

    loss_meter = AvgMeter()
    transformer_model.train()

    bar = tqdm(train_dataloader, total=math.ceil(len(train)/batch_size))

    for idx, data in enumerate(bar):

        source_ = data["src"].to(device)
        target_ = data["tgt"].to(device)

        count = source_.shape[0]

        output = transformer_model(source_, target_[:,:-1])

        output = output.reshape(-1, output.shape[2])
        target_ = target_[:, 1:]
        target_ = target_.reshape(-1)

        optimizer.zero_grad()
        loss = criterion(output, target_)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(transformer_model.parameters(), max_norm=1)

        optimizer.step()

        if scheduler:
            scheduler.step()

        writer.add_scalar("Training loss", loss, global_step=step)
        step += 1

        loss_meter.update(loss.item(), count)
        bar.set_postfix(loss=loss_meter.avg, lr=get_lr(optimizer), step=step)

    # Example Generation (Greedy Decode)
    ex = test[random.randint(0, len(test))]
    sentence = ex['text']
    src_indexes = torch.tensor(text_transform_sou(sentence)).unsqueeze(0).to(device)
    mapped_sentence_idx = translate_seq(transformer_model, src_indexes, device=device, max_len=30)
    mapped_sentence = [tar_vocab.get_itos()[i] for i in mapped_sentence_idx]
    print(f"\nExample source data: \n{sentence}\n")
    print(f"Original SNOMED mapping : \n{ex['label']}\n" )
    print(f"Generated by model SNOMED mapping : \n{' '.join(mapped_sentence[1:-1])}\n")

    del src_indexes, ex, sentence, mapped_sentence_idx, mapped_sentence, checkpoint
    torch.cuda.empty_cache()
    _ = gc.collect()


[Epoch 1 / 30]


100%|██████████| 6171/6171 [06:18<00:00, 16.32it/s, loss=5.14, lr=2.59e-5, step=6171]



Example source data: 
scar lower

Original SNOMED mapping : 
scar

Generated by model SNOMED mapping : 
neoplasm of skin

[Epoch 2 / 30]


100%|██████████| 6171/6171 [05:58<00:00, 17.22it/s, loss=3.13, lr=9.83e-5, step=12342]



Example source data: 
angiosarcoma

Original SNOMED mapping : 
angiosarcoma

Generated by model SNOMED mapping : 
neoplasm of lung

[Epoch 3 / 30]


...

100%|██████████| 6171/6171 [06:00<00:00, 17.13it/s, loss=1.08, lr=0.00098, step=55539]



Example source data: 
kidney disease no

Original SNOMED mapping : 
disorder of kidney and/or ureter

Generated by model SNOMED mapping : 
disorder of the ankle and/or foot

[Epoch 10 / 30]


100%|██████████| 6171/6171 [05:59<00:00, 17.15it/s, loss=1.05, lr=0.001, step=61710]



Example source data: 
urethral discharge

Original SNOMED mapping : 
urethral discharge

Generated by model SNOMED mapping : 
urethral discharge

[Epoch 11 / 30]


 35%|███▍      | 2147/6171 [02:04<03:48, 17.58it/s, loss=0.933, lr=0.000999, step=63857]

In [ ]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, model, device, max_len=50):
    tgts = []
    preds = []
    for datum in tqdm(data):
        src = datum["text"]
        tgt = datum["label"]
        src_idx = torch.tensor(text_transform_sou(src)).unsqueeze(0).to(device)
        pred_tgt = translate_seq(model, src_idx, device, max_len)
        pred_tgt = pred_tgt[1:-1]
        pred_sent = [tar_vocab.get_itos()[i] for i in pred_tgt]
        preds.append(pred_sent)
        tgts.append([tokenizer_eng(tgt.lower())])
    return bleu_score(preds, tgts)

In [ ]:
bleu = calculate_bleu(test, transformer_model, device)
print("BLEU Score Achieved :", bleu)

100%|██████████| 17424/17424 [10:46<00:00, 26.96it/s]


BLEU Score Achieved : 0.6227594614028931
